In [4]:
import requests
import os
import pandas as pd
from dotenv import load_dotenv
import json # JSON 파싱 오류 처리를 위해 추가

# --- 1. 설정 및 API 키 로드 ---
load_dotenv()
MOLIT_API_KEY = os.getenv("MOLIT_API_KEY") # .env 파일에 MOLIT_API_KEY=인증키 형식으로 저장

# --- 2. 국토부 API 요청 정보 ---
BASE_URL = " http://stat.molit.go.kr/portal/openapi/service/rest/getList.do?key=0678d88e2630421a9b2d64a23f5ab643&form_id=2086&style_num=713&start_dt=2024&end_dt=2024"

# 요청 파라미터 (예제 기반)
# form_id: 통계표 ID (2086 = 미분양현황_종합)
# style_num: 서식 번호 (713)
# start_dt, end_dt: 조회 기간 (YYYY) -> 월별 데이터는 다른 파라미터 필요 가능성 있음
PARAMS = {
    "key": MOLIT_API_KEY,
    "form_id": "2086",      # 미분양현황_종합
    "style_num": "713",
    "start_dt": "2015",     # 시작 년도 (최소 5년치 요청)
    "end_dt": "2024",       # 종료 년도
    # "date_se": "M",       # (추측) 월별 데이터를 원할 경우 필요할 수 있는 파라미터
    # "start_month": "01",  # (추측)
    # "end_month": "12",    # (추측)
}

# --- 3. 데이터 가져오기 함수 ---
def get_molit_unsold_data(params):
    """국토부 통계누리 API에서 미분양 데이터를 가져와 DataFrame으로 반환"""

    if not params.get("key"):
        print("🚨 오류: MOLIT_API_KEY가 .env 파일에 설정되지 않았습니다.")
        return pd.DataFrame()

    try:
        print("국토부 통계누리 API 요청 중...")
        response = requests.get(BASE_URL, params=params)
        response.raise_for_status() # HTTP 오류 체크

        # 국토부 API는 JSON 형식이 아닐 수 있음 -> XML 가능성 높음
        # 일단 JSON으로 시도, 실패 시 다른 처리 필요
        try:
            data = response.json() # JSON 파싱 시도
        except json.JSONDecodeError:
            print("🚨 오류: 응답이 JSON 형식이 아닙니다. XML일 가능성이 높습니다.")
            print("--- 응답 내용 (첫 500자) ---")
            print(response.text[:500]) # 실제 응답 내용 확인
            return pd.DataFrame()

        # 정상 응답 확인 (예제 기반)
        status = data.get("result_status", {})
        if status.get("status_code") != "INFO-000":
            print(f"🚨 API 오류: {status.get('message')} (코드: {status.get('status_code')})")
            return pd.DataFrame()

        # 실제 데이터 추출 (예제 기반)
        form_list = data.get("result_data", {}).get("formList", [])
        if not form_list:
            print("데이터를 찾을 수 없습니다.")
            return pd.DataFrame()

        print("데이터 로드 성공. DataFrame 변환 중...")
        df = pd.DataFrame(form_list)
        
        # 컬럼 이름 및 데이터 타입 정리 (예제 기반)
        df.rename(columns={'date': '연도'}, inplace=True)
        # '미분양(12월기준)' 컬럼 이름이 유효하지 않을 수 있음 -> 실제 응답 확인 필요
        if '미분양(12월기준)' in df.columns:
            df['미분양'] = pd.to_numeric(df['미분양(12월기준)'], errors='coerce')
        else:
            print("경고: '미분양(12월기준)' 컬럼을 찾을 수 없습니다. 실제 컬럼명을 확인하세요.")
            # 다른 컬럼이 실제 미분양 값일 수 있음 (예: 'value', 'data' 등)
            # df['미분양'] = pd.to_numeric(df['실제_미분양_컬럼명'], errors='coerce')
            
        df['연도'] = df['연도'].astype(str) # 연도 데이터를 문자열로 유지 (월별 데이터 아님)

        # 필요한 컬럼만 선택하고 정렬 (예제 기반)
        final_df = df[['연도', '대분류', '구분', '미분양']].sort_values(by=['연도', '대분류', '구분'])

        return final_df

    except requests.exceptions.RequestException as e:
        print(f"🚨 API 요청 중 네트워크 오류 발생: {e}")
    except Exception as e:
        print(f"🚨 처리 중 예상치 못한 오류 발생: {e}")
        
    return pd.DataFrame()

# --- 4. 메인 실행 ---
if __name__ == "__main__":
    
    unsold_data = get_molit_unsold_data(PARAMS)
    
    if not unsold_data.empty:
        print("\n--- 국토부 미분양 현황 데이터 (샘플) ---")
        print(unsold_data.head(10)) # 상위 10개 출력
        print("\n...")
        print(unsold_data.tail(10)) # 하위 10개 출력
    else:
        print("\n데이터 로드에 실패했습니다.")

국토부 통계누리 API 요청 중...
🚨 API 요청 중 네트워크 오류 발생: 500 Server Error: Internal Server Error for url: https://stat.molit.go.kr/portal/openapi/service/rest/getList.do?key=0678d88e2630421a9b2d64a23f5ab643&form_id=2086&style_num=713&start_dt=2024&end_dt=2024&key=0678d88e2630421a9b2d64a23f5ab643+%23+%EA%B5%AD%ED%86%A0%EB%B6%80+%ED%86%B5%EA%B3%84%EB%88%84%EB%A6%AC&form_id=2086&style_num=713&start_dt=2015&end_dt=2024

데이터 로드에 실패했습니다.


In [4]:
import requests  # 인터넷으로 데이터를 요청(request)하기 위해 필요한 도구
import json      # 받아온 데이터를 보기 좋게 정렬하기 위해 필요한 도구

# 1. API 요청에 필요한 정보들
api_key = "0678d88e2630421a9b2d64a23f5ab643"
form_id = "618"    # 통계표 ID (주택건설 인허가실적 총괄)
start_date = "2024"
end_date = "2024"

# 2. API를 호출할 주소(URL) 만들기
url = f" http://stat.molit.go.kr/portal/openapi/service/rest/getList.do?key=0678d88e2630421a9b2d64a23f5ab643&form_id=618&style_num=40&start_dt=2014&end_dt=2024"

print(f"'{url}' 주소로 데이터를 요청합니다...")
print("-" * 30)

# 3. requests를 사용해 인터넷으로 데이터 요청
response = requests.get(url)

# 4. 요청 결과 확인 (200이면 "성공")
if response.status_code == 200:
    print("✅ 데이터 요청 성공!")
    
    # 5. 받아온 응답을 JSON 형태로 변환
    data = response.json()
    
    # 6. 필요한 데이터(formList)만 뽑아내기
    data_list = data['result_data']['formList']
    
    # 7. 결과 출력
    print("\n--- [받아온 데이터 목록] ---")
    # json.dumps를 쓰면 딕셔너리를 보기 좋게 출력
    print(json.dumps(data_list, indent=2, ensure_ascii=False))

else:
    # 200이 아니면 "실패"
    print(f"❌ 데이터 요청 실패... (에러 코드: {response.status_code})")

' http://stat.molit.go.kr/portal/openapi/service/rest/getList.do?key=0678d88e2630421a9b2d64a23f5ab643&form_id=618&style_num=40&start_dt=2014&end_dt=2024' 주소로 데이터를 요청합니다...
------------------------------
✅ 데이터 요청 성공!

--- [받아온 데이터 목록] ---
[
  {
    "date": "2014",
    "합계": 529100,
    "구분1": "합계",
    "구분2": "합계"
  },
  {
    "date": "2014",
    "합계": 0,
    "구분1": "공공주택",
    "구분2": "국가기관"
  },
  {
    "date": "2014",
    "합계": 129,
    "구분1": "공공주택",
    "구분2": "지자체"
  },
  {
    "date": "2014",
    "합계": 20176,
    "구분1": "공공주택",
    "구분2": "주택사업자"
  },
  {
    "date": "2014",
    "합계": 0,
    "구분1": "공공주택",
    "구분2": "기타"
  },
  {
    "date": "2014",
    "합계": 34123,
    "구분1": "공공주택",
    "구분2": "소계"
  },
  {
    "date": "2014",
    "합계": 13818,
    "구분1": "공공주택",
    "구분2": "LH"
  },
  {
    "date": "2014",
    "합계": 494977,
    "구분1": "민간주택",
    "구분2": "소계"
  },
  {
    "date": "2014",
    "합계": 494977,
    "구분1": "민간주택",
    "구분2": "주택사업자등"
  },
  {
    "date": "2014",
    "합